### Do the imports
- If I am doing any processing of files in a folder I like to import os and glob
- import cellpose models for training of inferance
- import something to read your images 
- import libraries to save your results


### Install pytorch

Use this website to configure the right installation packages:
[Pytorch](https://pytorch.org/get-started/locally/)

then use `pip install <list of packages>` to install it

For my macbook, it is

```pip3 install torch torchvision torchaudio```



In [15]:
import os
import glob

import tifffile # read an write tiff files
from cellpose import models
import napari
import torch

import utils

In [2]:
viewer = napari.Viewer()

In [10]:
if not os.path.exists("Data"):
    url = "https://www.dropbox.com/s/oga2fhdxbgq2k3h/Data.zip?dl=1"
    utils.download_images(url, ".")
else:
    print("Data already exists")

In [11]:
imagefiles = sorted(glob.glob("Data/*.tif"))
imagefiles

['Data/a01_42.tif', 'Data/a01_43.tif']

In [12]:
x = tifffile.imread("Data/a01_42.tif")
x.shape

(4, 2160, 2160)

In [13]:
viewer.add_image(x, channel_axis=0)

[<Image layer 'Image' at 0x2f6a64f40>,
 <Image layer 'Image [1]' at 0x2f005fd60>,
 <Image layer 'Image [2]' at 0x2f6fb8ac0>,
 <Image layer 'Image [3]' at 0x2fc846350>]

### Setup the cellpose model

#### GPU setting
- on windows or linux with an NVIDIA GPU, set `gpu=True``
- on M1 or M2 mac, set `device=torch.device('mps')`
- on old Mac or Windows without GPU, set `gpu=False` -- this will be slower

#### Model
- cellpose has a number of pretrained models to use. Start with `cyto` (even it is nuclei)
- Try 'nuclei', 'cyto2', if cyto doesn't work.

In [16]:
d = torch.device('mps')
model = models.Cellpose(gpu=False, device=d, model_type='cyto')

### `model.eval` options

The basic options:
- x : the image, can be a 2d numpy array, a list of numpy arrays,
or a 3d numpy array
- diameter : The approximate size of the object you are trying to segment
- channels :
    - [0, 0] for a grayscale image
    - [] 

In [59]:
masks, flows, styles, diams = model.eval(x[3], channels=[0, 0],   #[0, 3] might work best
                                         diameter=25,
                                         cellprob_threshold=-2,
                                         flow_threshold=.8)

In [54]:
masks1, flows1, styles1, diams1 = model.eval(x[[0, 3], ...], channels=[0, 1],   #[0, 3] might work best
                                         diameter=25,
                                         cellprob_threshold=-2,
                                         flow_threshold=.8)

In [60]:
viewer.add_labels(masks)
viewer.add_labels(masks1)

<Labels layer 'masks1' at 0x390da4310>

In [25]:
viewer.add_image(flows[0])

<Image layer 'Image [4]' at 0x302292ec0>

In [26]:
viewer.add_image(flows[-2])

<Image layer 'Image [5]' at 0x38b2efd00>

In [61]:
masks1.max(), masks.max() 

(3982, 3974)